# Установка необходимых библиотек

In [31]:
import sberpm
import graphviz

print("Версия SberPM ", sberpm.__version__)

Версия SberPM  3.4.0


In [32]:
import pandas as pd
import numpy as np
from sberpm import DataHolder
from sberpm.miners import SimpleMiner
from sberpm.metrics import ActivityMetric, TransitionMetric
from sberpm.visual import GraphvizPainter
from sberpm.autoinsights import AutoInsights
from sberpm.imitation import Simulation
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Предобработка и анализ данных

In [33]:
def preprocess_data_with_new_stage(filepath: str) -> DataHolder:
    df = pd.read_csv(filepath, sep=";", encoding="utf-8")

    required_columns = ['case_id', 'line_id', 'from', 'to', 'time']
    if not all(column in df.columns for column in required_columns):
        raise ValueError(f"Отсутствуют обязательные колонки. Нужны: {required_columns}")

    df.drop_duplicates(inplace=True)
    df.dropna(subset=['case_id', 'from', 'to', 'time'], inplace=True)
    df = df[df['time'] > 0]

    # Временная разметка
    start_datetime = datetime(2024, 1, 1, 0, 0, 0)
    df['start_time'] = None
    df['end_time'] = None

    for case in df['case_id'].unique():
        case_mask = df['case_id'] == case
        case_df = df[case_mask].sort_values(by='line_id')

        start_time = start_datetime
        start_times = []
        end_times = []

        for time in case_df['time']:
            start_times.append(start_time.strftime("%Y-%m-%d %H:%M:%S"))
            end_time = start_time + timedelta(hours=time)
            end_times.append(end_time.strftime("%Y-%m-%d %H:%M:%S"))
            start_time = end_time

        df.loc[case_mask, 'start_time'] = start_times
        df.loc[case_mask, 'end_time'] = end_times

    # Случайный сдвиг начала кейса
    case_offsets = {case: timedelta(hours=np.random.uniform(0.01, 0.1)) for case in df['case_id'].unique()}
    df['start_time'] = pd.to_datetime(df['start_time']) + df['case_id'].map(case_offsets)
    df['end_time'] = pd.to_datetime(df['end_time']) + df['case_id'].map(case_offsets)
    df.to_csv("df.csv", index=False)

    # Создание DataHolder
    data_holder = DataHolder(
        data=df,
        col_case='case_id',
        col_stage='to',
        col_start_time='start_time',
        col_end_time='end_time',
        col_duration='time',
        time_format="%Y-%m-%d %H:%M:%S"
    )
    
    return data_holder

In [34]:
data_holder = preprocess_data_with_new_stage("HR_log_obezlich.csv")
data_holder.data.head()

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



,case_id,line_id,from,to,time,start_time,end_time
0,1,1,Новый,Рассмотрение заказчиком,184.081131,2024-01-01 00:01:05,2024-01-08 16:05:57
1,1,2,Рассмотрение заказчиком,Оценка кандидата,216.771912,2024-01-08 16:05:57,2024-01-17 16:52:15
2,1,3,Оценка кандидата,Отклонен,222.700653,2024-01-17 16:52:15,2024-01-26 23:34:18
3,2,1,Новый,Рассмотрение заказчиком,528.787109,2024-01-01 00:02:24,2024-01-23 00:49:37
4,2,2,Рассмотрение заказчиком,Интервью,104.049965,2024-01-23 00:49:37,2024-01-27 08:52:37


# Process Mining исследование с использованием библиотеки sberpm

In [35]:
activity_metric = ActivityMetric(data_holder, time_unit="h")
nodes_count_metric = activity_metric.count().to_dict()
nodes_mean_metric = activity_metric.mean_duration().to_dict()

simple_miner = SimpleMiner(data_holder)
simple_miner.apply()
graph = simple_miner.graph

graph.add_node_metric('count', nodes_count_metric)
graph.add_node_metric('mean_duration', nodes_mean_metric)

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

In [36]:
def analyze_process(data_holder):
    
    # Метрика активностей
    activity_metric = ActivityMetric(data_holder, time_unit='h').apply().reset_index()
    activity_cols = ['index', 'count', 'unique_ids_num', 'aver_count_in_trace', 'loop_percent', 'throughput']
    print("\nТоп-5 активностей по частоте выполнения и зацикленности:")
    display(activity_metric[activity_cols]
            .sort_values(by=['count', 'loop_percent'], ascending=[False, False])
            .head()
            .reset_index(drop=True))

    # Метрика переходов
    transition_metric = TransitionMetric(data_holder, time_unit='h').apply().reset_index()
    transition_cols = ['index', 'count', 'unique_ids_num', 'aver_count_in_trace', 'loop_percent', 'throughput']
    print("\nТоп-10 переходов по частоте выполнения и зацикленности:")
    display(transition_metric[transition_cols]
            .sort_values(by=['count', 'loop_percent'], ascending=[False, False])
            .head(10)
            .reset_index(drop=True))

    return {
        'activity_metric': activity_metric,
        'transition_metric': transition_metric
    }

In [37]:
analysis_results = analyze_process(data_holder)


Топ-5 активностей по частоте выполнения и зацикленности:


,index,count,unique_ids_num,aver_count_in_trace,loop_percent,throughput
0,Интервью,814,284,2.866197,65.110565,20.450053
1,Отклонен,768,768,1.000000,0.000000,8.733939
2,Рассмотрение заказчиком,569,569,1.000000,0.000000,10.620900
3,Рассмотрение рекрутером,510,494,1.032389,3.137255,7.879697
4,Заполнение анкеты,417,201,2.074627,51.798561,40.244595



Топ-10 переходов по частоте выполнения и зацикленности:


,index,count,unique_ids_num,aver_count_in_trace,loop_percent,throughput
0,"(Интервью, Интервью)",426,169,2.520710,60.328638,19.121051
1,"(Оценка кандидата, Отклонен)",252,252,1.000000,0.000000,11.242193
2,"(Заполнение анкеты, Заполнение анкеты)",216,110,1.963636,49.074074,42.782205
3,"(Планирование интервью, Интервью)",213,163,1.306748,23.474178,55.374194
4,"(Планирование интервью, Отклонен)",203,203,1.000000,0.000000,28.887048
5,"(Интервью, Заполнение анкеты)",201,201,1.000000,0.000000,20.253121
6,"(Заполнение анкеты, Проверка)",201,201,1.000000,0.000000,37.833077
7,"(Проверка, Решение о найме)",201,201,1.000000,0.000000,20.086284
8,"(Решение о найме, Согласование офера в компании)",201,201,1.000000,0.000000,8.910508
9,"(Согласование офера в компании, Готов к оформл...",201,201,1.000000,0.000000,41.968472


In [38]:
from sberpm.autoinsights import AutoInsights

auto_insights = AutoInsights(data_holder)

auto_insights.apply()

bool_insights = auto_insights.bool_insights()
print("Проблемные этапы (True - есть проблема):")
display(bool_insights)

float_insights = auto_insights.float_insights()
print("Оценка серьёзности проблем:")
display(float_insights)

summary = auto_insights.fin_effects_summary()
print("Выявленные проблемы в процессе найма:")
print(summary)

Проблемные этапы (True - есть проблема):


Метрика                       Длительность операции              \
Операция                         Растет со временем Bottle neck   
Рассмотрение заказчиком                       False       False   
Оценка кандидата                               True       False   
Отклонен                                      False       False   
Интервью                                       True       False   
Заполнение анкеты                              True       False   
Проверка                                       True       False   
Решение о найме                               False       False   
Согласование офера в компании                 False       False   
Готов к оформлению                             True       False   
Оформление                                     True       False   
Оформлен                                      False       False   
Планирование интервью                         False       False   
Рассмотрение рекрутером                       False       False   
Готов к переводу                              False       False   
Не выходит на связь                           False       False   
Резерв                                         True       False   
Самоотказ                                      True       False   

Метрика                                                                  \
Операция                      Нестандартизированная или ручная операция   
Рассмотрение заказчиком                                           False   
Оценка кандидата                                                  False   
Отклонен                                                          False   
Интервью                                                          False   
Заполнение анкеты                                                 False   
Проверка                                                          False   
Решение о найме                                                   False   
Согласование офера в компании                                     False   
Готов к оформлению                                                False   
Оформление                                                        False   
Оформлен                                                          False   
Планирование интервью                                             False   
Рассмотрение рекрутером                                           False   
Готов к переводу                                                  False   
Не выходит на связь                                               False   
Резерв                                                            False   
Самоотказ                                                         False   

Метрика                                                                 \
Операция                      Разовые инциденты Многократные инциденты   
Рассмотрение заказчиком                   False                  False   
Оценка кандидата                          False                  False   
Отклонен                                  False                  False   
Интервью                                  False                  False   
Заполнение анкеты                         False                  False   
Проверка                                  False                  False   
Решение о найме                           False                  False   
Согласование офера в компании             False                  False   
Готов к оформлению                        False                  False   
Оформление                                False                  False   
Оформлен                                  False                  False   
Планирование интервью                     False                  False   
Рассмотрение рекрутером                   False                  False   
Готов к переводу                          False                  False   
Не выходит на связь                       False                  False   
Резерв                                    

Оценка серьёзности проблем:


Метрика                       Длительность операции              \
Операция                         Растет со временем Bottle neck   
Рассмотрение заказчиком                    0.988748    0.459356   
Оценка кандидата                           1.000000    0.000000   
Отклонен                                   0.995688    1.000000   
Интервью                                   0.995780    0.434636   
Заполнение анкеты                          0.995781    0.421379   
Проверка                                   0.995862    0.000000   
Решение о найме                            0.993885    0.000000   
Согласование офера в компании              0.995767    0.496747   
Готов к оформлению                         0.996361    0.000000   
Оформление                                 0.996116    0.507137   
Оформлен                                   0.995773    0.768538   
Планирование интервью                      0.995779    0.586357   
Рассмотрение рекрутером                    0.000000    0.986984   
Готов к переводу                           0.995752    0.521946   
Не выходит на связь                        0.995779    0.565154   
Резерв                                     0.995798    0.564622   
Самоотказ                                  0.995789    0.624677   

Метрика                                                                  \
Операция                      Нестандартизированная или ручная операция   
Рассмотрение заказчиком                                        0.952314   
Оценка кандидата                                               0.000000   
Отклонен                                                       0.273456   
Интервью                                                       0.983355   
Заполнение анкеты                                              1.000000   
Проверка                                                       0.000000   
Решение о найме                                                0.000000   
Согласование офера в компании                                  0.905365   
Готов к оформлению                                             0.000000   
Оформление                                                     0.892319   
Оформлен                                                       0.564091   
Планирование интервью                                          0.792846   
Рассмотрение рекрутером                                        0.289800   
Готов к переводу                                               0.873723   
Не выходит на связь                                            0.819470   
Резерв                                                         0.820138   
Самоотказ                                                      0.744730   

Метрика                                                                 \
Операция                      Разовые инциденты Многократные инциденты   
Рассмотрение заказчиком                0.134950               0.459356   
Оценка кандидата                       0.068269               0.000000   
Отклонен                               0.061983               1.000000   
Интервью                               0.143937               0.434636   
Заполнение анкеты                      1.000000               0.421379   
Проверка                               0.367450               0.000000   
Решение о найме                        0.000000               0.000000   
Согласование офера в компании          0.053714               0.496747   
Готов к оформлению                     0.280407               0.000000   
Оформление                             0.066383               0.507137   
Оформлен                               0.146954               0.768538   
Планирование интервью                  0.298137               0.586357   
Рассмотрение рекрутером                0.409745               0.986984   
Готов к переводу                       0.062604               0.521946   
Не выходит на связь                    0.359719               0.565154   
Резерв                                 0.7

Выявленные проблемы в процессе найма:

                    Длительность следующих этапов увеличивается со временем, что может привести в дальнейшем к проблемам в процессе: «Оценка кандидата», «Интервью», «Заполнение анкеты», «Проверка», «Готов к оформлению», «Оформление», «Резерв», «Самоотказ».

                
                    Следующие этапы являются нерегулярными (редкими) и не требуются для успешной реализации процесса: «Готов к переводу», «Не выходит на связь», «Резерв». Максимальный потенциальный финансовый эффект при отказе от данных этапов 1617,66 рублей.

                
                    На следующих этапах наблюдается зацикленность, при которой экземпляр процесса начинается и заканчивается на один и тот же этап: «Рассмотрение заказчиком», «Оценка кандидата», «Отклонен», «Интервью», «Заполнение анкеты», «Проверка», «Решение о найме», «Согласование офера в компании», «Готов к оформлению», «Оформление», «Оформлен», «Планирование интервью», «Рассмотрение рекрутером», «Гот

In [39]:
from sberpm.visual import Graph

def show_graph(graph: Graph, save=True, filename="graph", **kwargs):
    if kwargs.get("happy_path"):
        painter.apply_happy_path(graph, kwargs["happy_path"])
    elif kwargs.get("auto_insights"):
        painter.apply_insights(graph, **kwargs["auto_insights"])
    else:
        painter.apply(graph, **kwargs)

    display(painter.show())

In [40]:
insights_miner = SimpleMiner(data_holder)
insights_miner.apply()

insights_graph = insights_miner.graph

transition_insights = auto_insights.autoinsights_for_transitions()
transition_insights.apply()

show_graph(
    insights_graph,
    filename="AutoInsights",
    auto_insights=dict(
        insight_activity_obj=auto_insights,
        insight_transition_obj=transition_insights,
    ),
)

In [41]:
def detect_anomalous_stages(activity_metric, threshold_count=100, threshold_throughput=1.0, threshold_loop=50):

    # Фильтрация по условиям
    rare = activity_metric[activity_metric['count'] < threshold_count]
    slow = activity_metric[activity_metric['throughput'] < threshold_throughput]
    loopy = activity_metric[activity_metric['loop_percent'] > threshold_loop]

    # Объединяем все уникальные аномалии
    anomalous_stages = pd.concat([rare, slow, loopy])['index'].unique().tolist()

    print(f"Найдено аномальных этапов: {len(anomalous_stages)}")
    print("Аномальные этапы:", anomalous_stages)

    return anomalous_stages

In [42]:
activity_df = ActivityMetric(data_holder, time_unit='h').apply().reset_index()
anomalous_stages = detect_anomalous_stages(activity_df)

successful_cases = data_holder.data.groupby("case_id")["to"].apply(set)
cases_without_anomalies = successful_cases.apply(lambda stages: all(stage not in stages for stage in anomalous_stages))

successful_without_anomalies = successful_cases[cases_without_anomalies]
print("Количество успешных экземпляров без аномальных этапов:", len(successful_without_anomalies))

Найдено аномальных этапов: 6
Аномальные этапы: ['Самоотказ', 'Не выходит на связь', 'Готов к переводу', 'Резерв', 'Интервью', 'Заполнение анкеты']
Количество успешных экземпляров без аномальных этапов: 495


# Ручная симуляция процесса с помощью библиотеки sberpm

In [43]:
def generate_and_prepare_holder(simulation, iterations=1000):
    simulation.generate(iterations=iterations)
    data = simulation.get_result()
    return data, DataHolder(
        data=data,
        col_case="case_id",
        col_stage="to",
        col_start_time="start_time",
        col_end_time="end_time",
        col_duration="time",
        time_format="%Y-%m-%d %H:%M:%S"
    )

def analyze_and_show(holder, title="graph", label=""):
    miner = SimpleMiner(holder)
    miner.apply()

    activity_metric = ActivityMetric(holder, time_unit="h")
    nodes_count_metric = activity_metric.count().to_dict()
    nodes_mean_metric = activity_metric.mean_duration().to_dict()
    
    graph = miner.graph
    graph.add_node_metric('count', nodes_count_metric)
    graph.add_node_metric('mean_duration', nodes_mean_metric)
    
    show_graph(graph, filename=title, node_style_metric="mean_duration", edge_style_metric="mean_duration")
    
    avg_time = holder.data["time"].mean()
    print(f"{label} средняя длительность процесса: {avg_time:.2f} часов")

    avg_case_duration = holder.data.groupby("case_id")["time"].sum().mean()
    print(f"Средняя длительность процесса (по кейсу): {avg_case_duration:.2f} часов")

    successful_case_ids = holder.data[holder.data['to'] == 'Оформлен']['case_id'].unique()
    successful_cases_duration = holder.data[holder.data['case_id'].isin(successful_case_ids)]
    real_success_avg = successful_cases_duration.groupby('case_id')['time'].sum().mean()
    
    print(f"Средняя длительность успешных кейсов (до 'Оформлен'): {real_success_avg:.2f} часов")

    return avg_case_duration

In [44]:
simulation = Simulation(data_holder)

# Генерация и анализ As-Is
as_is_data, as_is_holder = generate_and_prepare_holder(simulation, iterations=1000)
as_is_duration = analyze_and_show(as_is_holder, title="as_is_graph", label="As-Is")

  0%|          | 0/999 [00:00<?, ?it/s]

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



As-Is средняя длительность процесса: 341.67 часов
Средняя длительность процесса (по кейсу): 1812.93 часов
Средняя длительность успешных кейсов (до 'Оформлен'): 2380.44 часов


In [45]:
# --- Ускорение ключевых тормозящих стадий ---
for stage in [
    "Оценка кандидата", "Интервью", "Заполнение анкеты", "Проверка",
]:
    simulation.scale_time_node(stage, scale=0.5)

# --- Удаление редких стадий ---
for stage in ["Не выходит на связь", "Резерв", "Самоотказ"]:
    simulation.delete_node(stage)

# --- Удаление циклов в себя ---
for stage in ["Интервью", "Заполнение анкеты", "Планирование интервью"]:
    simulation.delete_loop(stage)

# --- Удаление переходов ---
for edge in [
    ("Планирование интервью", "Рассмотрение рекрутером"),
    ("Интервью", "Планирование интервью"),
]:
    simulation.delete_edge(*edge)


# Генерация и анализ оптимизированного вручную процесса
what_if_data, what_if_holder = generate_and_prepare_holder(simulation, iterations=1000)
what_if_duration = analyze_and_show(what_if_holder, title="what_if_graph", label="Ручная оптимизация")

  0%|          | 0/999 [00:00<?, ?it/s]

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



Ручная оптимизация средняя длительность процесса: 263.93 часов
Средняя длительность процесса (по кейсу): 1230.98 часов
Средняя длительность успешных кейсов (до 'Оформлен'): 1474.32 часов


In [46]:
print("Среднее количество шагов в кейсе (до):", as_is_holder.data.groupby("case_id").size().mean())
print("Среднее количество шагов в кейсе (после):", what_if_holder.data.groupby("case_id").size().mean())

Среднее количество шагов в кейсе (до): 5.306
Среднее количество шагов в кейсе (после): 4.664


# RL методы (Qlearning, CrossEntropy, Genetic)

## Дополнительная аналитика

In [47]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [48]:
import networkx as nx
from itertools import combinations
from datetime import datetime, timedelta

# Добавляем первоначальное состояние (Новый)
def add_new(df):
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])

    new_rows = []
    for case_id in df['case_id'].unique():
        min_row = df[df['case_id'] == case_id].sort_values('start_time').iloc[0]
        new_row = {
            'case_id': case_id,
            'line_id': -1,
            'from': 'START',
            'to': 'Новый',
            'time': 0.0,
            'start_time': min_row['start_time'] - timedelta(seconds=1),
            'end_time': min_row['start_time'] - timedelta(seconds=1),
        }
        new_rows.append(new_row)

    df_with_new = pd.concat([pd.DataFrame(new_rows), df], ignore_index=True).sort_values(['case_id', 'start_time'])

    df_with_new['start_time'] = df_with_new['start_time'].dt.strftime("%Y-%m-%d %H:%M:%S")
    df_with_new['end_time'] = df_with_new['end_time'].dt.strftime("%Y-%m-%d %H:%M:%S")

    df_with_new.to_csv("df_with_new.csv", index=False)

    return df_with_new

df = add_new(pd.read_csv("df.csv"))

transitions = df.groupby(['from', 'to'])['time'].mean().reset_index()
G = nx.DiGraph()

for _, row in transitions.iterrows():
    src, dst, avg_time = row['from'], row['to'], row['time']
    G.add_edge(src, dst, weight=avg_time)

required_stages = [
    'Рассмотрение рекрутером',
    'Рассмотрение заказчиком',
    'Оценка кандидата',
    'Интервью',
    'Решение о найме',
    'Готов к оформлению'
]

# Функция для поиска лучшего маршрута через максимально достижимые стадии
def find_best_path(graph, required_stages, start='Новый', end='Оформлен'):
    best_combination = None
    best_path = []
    best_time = float('inf')

    for r in range(len(required_stages), 2, -1):
        for subset in combinations(required_stages, r):
            try:
                stages = [start] + list(subset) + [end]
                path = []
                total_time = 0
                valid = True
                for i in range(len(stages) - 1):
                    sub_path = nx.shortest_path(graph, stages[i], stages[i + 1], weight='weight')
                    if path:
                        sub_path = sub_path[1:]  # исключаем дубли
                    path += sub_path
                    for j in range(len(sub_path) - 1):
                        total_time += graph[sub_path[j]][sub_path[j + 1]]['weight']
            except nx.NetworkXNoPath:
                valid = False
            if valid and total_time < best_time:
                best_combination = subset
                best_path = path
                best_time = total_time
    return best_path, best_combination, best_time

path, coverage, total_time = find_best_path(G, required_stages)
print("Рекомендованный путь:")
for stage in path:
    print(f" → {stage}")

print(f"\nПокрытые ключевые стадии ({len(coverage)} из {len(required_stages)}):")
print(", ".join(coverage))

print(f"\nОбщая длительность: {total_time:.2f}")

Рекомендованный путь:
 → Новый
 → Рассмотрение рекрутером
 → Рассмотрение заказчиком
 → Интервью
 → Заполнение анкеты
 → Проверка
 → Решение о найме
 → Согласование офера в компании
 → Готов к оформлению
 → Оформление
 → Оформлен

Покрытые ключевые стадии (5 из 6):
Рассмотрение рекрутером, Рассмотрение заказчиком, Интервью, Решение о найме, Готов к оформлению

Общая длительность: 1106.02


In [49]:
avg_duration = df.groupby('case_id')['time'].sum().mean()
print(f"Реальная средняя длительность одного кейса: {avg_duration:.2f}")

Реальная средняя длительность одного кейса: 1399.35


In [50]:
successful_case_ids = df[df['to'] == 'Оформлен']['case_id'].unique()
successful_cases_duration = df[df['case_id'].isin(successful_case_ids)]
real_success_avg = successful_cases_duration.groupby('case_id')['time'].sum().mean()

print(f"Средняя длительность успешных кейсов (до 'Оформлен'): {real_success_avg:.2f}")

Средняя длительность успешных кейсов (до 'Оформлен'): 1979.98


In [51]:
import random
from collections import defaultdict

def load_data(path):
    df = pd.read_csv(path)
    df = df[df["from"] != "START"]
    return df

# --- Метрики ---
def calculate_success_rate(paths, success_state="Оформлен"):
    return round(sum(1 for p, _ in paths if p[-1] == success_state) / len(paths) * 100, 2)

def calculate_avg_path_length(paths):
    return round(np.mean([len(p) for p, _ in paths]), 2)

def calculate_avg_time(paths):
    return round(np.mean([t for _, t in paths]), 2)

def calculate_avg_time_successful(paths, transition_times=None, success_state="Оформлен"):
    if transition_times:
        success_paths = [p for p, _ in paths if p[-1] == success_state]
        times = [sum(transition_times.get((p[i], p[i+1]), 0) for i in range(len(p)-1)) for p in success_paths]
        return round(np.mean(times), 2) if times else None
    else:
        success_times = [t for path, t in paths if path[-1] == success_state]
        return round(np.mean(success_times), 2) if success_times else None

## Qlearning

In [52]:
# --- Q-Learning (базовый) ---
class QLearningSimulator:
    def __init__(self, transitions_df):
        self.transitions = transitions_df
        self.state_actions = defaultdict(list)
        self.rewards = defaultdict(dict)
        self._prepare_environment()

    def _prepare_environment(self):
        for _, row in self.transitions.iterrows():
            self.state_actions[row["from"]].append(row["to"])
            self.rewards[row["from"]][row["to"]] = -row["time"]
        self.terminal_states = ["Отклонен", "Оформлен", "Самоотказ", "Не выходит на связь"]

    def reset(self):
        self.current_state = "Новый"
        return self.current_state

    def step(self, action):
        if action not in self.state_actions.get(self.current_state, []):
            return self.current_state, -100, True
        reward = self.rewards[self.current_state][action]
        done = action in self.terminal_states
        self.current_state = action
        return action, reward, done

    def train_q_learning(self, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
        Q = defaultdict(lambda: defaultdict(float))
        for _ in range(episodes):
            state = self.reset()
            for _ in range(50):
                actions = self.state_actions.get(state, [])
                if not actions:
                    break
                action = random.choice(actions) if random.random() < epsilon else max(actions, key=lambda a: Q[state][a])
                next_state, reward, done = self.step(action)
                max_q = max([Q[next_state][a] for a in self.state_actions.get(next_state, [])], default=0)
                Q[state][action] += alpha * (reward + gamma * max_q - Q[state][action])
                state = next_state
                if done:
                    break
        return Q

# --- Q-Learning с усиленными наградами ---
class QLearningSimulatorEnhanced(QLearningSimulator):
    def _prepare_environment(self):
        for _, row in self.transitions.iterrows():
            f, t, time = row["from"], row["to"], row["time"]
            reward = -time
            if t == "Оформлен":
                reward += 500
            elif t == "Решение о найме":
                reward += 100
            elif t == "Проверка":
                reward += 50
            elif t in ["Самоотказ", "Не выходит на связь"]:
                reward -= 200
            elif t == "Отклонен":
                reward -= 300
            self.state_actions[f].append(t)
            self.rewards[f][t] = reward
        self.terminal_states = ["Отклонен", "Оформлен", "Самоотказ", "Не выходит на связь"]

def simulate_q_policy(Q, transitions_df, terminal_states, n_simulations=100):
    state_actions = defaultdict(list)
    rewards = defaultdict(dict)
    for _, row in transitions_df.iterrows():
        state_actions[row["from"]].append(row["to"])
        rewards[row["from"]][row["to"]] = -row["time"]
    paths = []
    for _ in range(n_simulations):
        state = "Новый"
        path = [state]
        total_reward = 0
        for _ in range(50):
            actions = state_actions.get(state, [])
            if not actions:
                break
            action = max(Q[state], key=Q[state].get) if state in Q and Q[state] else random.choice(actions)
            reward = rewards[state][action]
            path.append(action)
            total_reward += reward
            if action in terminal_states:
                break
            state = action
        paths.append((path, -total_reward))
    return paths

def extract_best_policy(Q, min_q_threshold=0.0):
    print("\n--- Strategy ---")
    for state in Q:
        if Q[state]:
            best_action = max(Q[state], key=Q[state].get)
            q_value = Q[state][best_action]
            if q_value > min_q_threshold:
                print(f"{state} → {best_action} | Prob = {q_value:.2f}")

In [53]:
path = "df_with_new.csv"
df = load_data(path)
terminal_states = ["Отклонен", "Оформлен", "Самоотказ", "Не выходит на связь"]

# До гипотез
q_env = QLearningSimulator(df)
q_table = q_env.train_q_learning()
q_paths = simulate_q_policy(q_table, df, terminal_states)

print("\n--- BEFORE HYPOTHESES ---")
print("Success rate:", calculate_success_rate(q_paths))
print("Average time:", calculate_avg_time(q_paths))
print("Avg path length:", calculate_avg_path_length(q_paths))
print("Average time (successful only):", calculate_avg_time_successful(q_paths))
extract_best_policy(q_table)

# После гипотез
hyp_df = df.copy()
hyp_df = hyp_df[~((hyp_df["from"] == "Рассмотрение рекрутером") & (hyp_df["to"] == "Отклонен"))]
hyp_df = hyp_df[~((hyp_df["from"] == "Интервью") & (hyp_df["to"] == "Интервью"))]
hyp_df = hyp_df[(hyp_df["from"] != "Оценка кандидата") & (hyp_df["to"] != "Оценка кандидата")]

q_enhanced_env = QLearningSimulatorEnhanced(hyp_df)
q_table_mod = q_enhanced_env.train_q_learning()
q_paths_mod = simulate_q_policy(q_table_mod, hyp_df, terminal_states)

print("\n--- AFTER HYPOTHESES + REWARDS ---")
print("Success rate:", calculate_success_rate(q_paths_mod))
print("Average time:", calculate_avg_time(q_paths_mod))
print("Avg path length:", calculate_avg_path_length(q_paths_mod))
print("Average time (successful only):", calculate_avg_time_successful(q_paths_mod))
extract_best_policy(q_table_mod)


--- BEFORE HYPOTHESES ---
Success rate: 0.0
Average time: 49.44
Avg path length: 3.0
Average time (successful only): None

--- Strategy ---

--- AFTER HYPOTHESES + REWARDS ---
Success rate: 100.0
Average time: 251.87
Avg path length: 4.0
Average time (successful only): 251.87

--- Strategy ---
Новый → Рассмотрение рекрутером | Prob = 191.69
Готов к оформлению → Оформление | Prob = 293.91
Оформление → Оформлен | Prob = 474.77
Рассмотрение рекрутером → Готов к переводу | Prob = 266.76
Готов к переводу → Оформлен | Prob = 297.67


## Cross Entropy

In [54]:
# --- Cross Entropy ---
def train_cross_entropy(transitions_df, terminal_states, reward_map=None, iterations=20, n_samples=200, elite_frac=0.2):
    transition_counts = transitions_df.groupby(["from", "to"]).size().reset_index(name="count")
    transition_counts["prob"] = transition_counts["count"] / transition_counts.groupby("from")["count"].transform("sum")
    probs_matrix = transition_counts.pivot(index="from", columns="to", values="prob").fillna(0)

    def simulate_paths(probs_matrix, n):
        paths = []
        for _ in range(n):
            state = "Новый"
            path = [state]
            total_reward = 0
            for _ in range(50):
                if state not in probs_matrix.index:
                    break
                probs = probs_matrix.loc[state]
                if probs.sum() == 0:
                    break
                next_state = np.random.choice(probs.index, p=probs / probs.sum())
                time = transitions_df[(transitions_df["from"] == state) & (transitions_df["to"] == next_state)]["time"].mean()
                reward = -time
                if reward_map:
                    reward += reward_map.get(next_state, 0)
                path.append(next_state)
                total_reward += reward
                if next_state in terminal_states:
                    break
                state = next_state
            paths.append((path, total_reward))
        return paths

    for _ in range(iterations):
        samples = simulate_paths(probs_matrix, n_samples)
        elite = sorted(samples, key=lambda x: -x[1])[:int(n_samples * elite_frac)]
        updated = defaultdict(lambda: defaultdict(int))
        for path, _ in elite:
            for i in range(len(path) - 1):
                updated[path[i]][path[i + 1]] += 1
        probs_matrix = pd.DataFrame([
            {"from": f, "to": t, "prob": count / sum(updated[f].values())}
            for f in updated for t, count in updated[f].items()
        ]).pivot(index="from", columns="to", values="prob").fillna(0)

    return probs_matrix

def simulate_cem_paths(probs_matrix, transitions_df, terminal_states, reward_map=None, n=100):
    paths = []
    for _ in range(n):
        state = "Новый"
        path = [state]
        total_time = 0
        for _ in range(50):
            if state not in probs_matrix.index:
                break
            probs = probs_matrix.loc[state]
            if probs.sum() == 0:
                break
            next_state = np.random.choice(probs.index, p=probs / probs.sum())
            time = transitions_df[(transitions_df["from"] == state) & (transitions_df["to"] == next_state)]["time"].mean()
            reward = -time
            if reward_map:
                reward += reward_map.get(next_state, 0)
            path.append(next_state)
            total_time += reward
            if next_state in terminal_states:
                break
            state = next_state
        paths.append((path, total_time))
    return paths

def extract_cem_policy(probs_matrix):
    print("\n--- Strategy ---")
    for state in probs_matrix.index:
        if probs_matrix.loc[state].max() > 0:
            best_action = probs_matrix.loc[state].idxmax()
            prob = probs_matrix.loc[state].max()
            print(f"{state} → {best_action} | Prob = {prob:.2f}")

In [55]:
reward_map = {
    "Оформлен": 500,
    "Решение о найме": 300,
    "Проверка": 50,
    "Отклонен": -300,
    "Самоотказ": -200,
    "Не выходит на связь": -200
}

# До гипотез
cem_probs = train_cross_entropy(df, terminal_states)
cem_paths = simulate_cem_paths(cem_probs, df, terminal_states)

print("\n--- BEFORE HYPOTHESES ---")
print("Success rate:", calculate_success_rate(cem_paths))
print("Average time (successful only):", calculate_avg_time_successful(cem_paths))
print("Avg path length:", calculate_avg_path_length(cem_paths))
extract_cem_policy(cem_probs)

# После гипотез
mod_df = df.copy()
mod_df = mod_df[~((mod_df["from"] == "Рассмотрение рекрутером") & (mod_df["to"] == "Не выходит на связь"))]

cem_probs_mod = train_cross_entropy(mod_df, terminal_states, reward_map=reward_map)
cem_paths_mod = simulate_cem_paths(cem_probs_mod, mod_df, terminal_states, reward_map=reward_map)

print("\n--- AFTER HYPOTHESES + REWARDS ---")
print("Success rate:", calculate_success_rate(cem_paths_mod))
print("Average time (successful only):", calculate_avg_time_successful(cem_paths_mod))
print("Avg path length:", calculate_avg_path_length(cem_paths_mod))
extract_cem_policy(cem_probs_mod)


--- BEFORE HYPOTHESES ---
Success rate: 0.0
Average time (successful only): None
Avg path length: 3.0

--- Strategy ---
Новый → Рассмотрение рекрутером | Prob = 1.00
Рассмотрение рекрутером → Не выходит на связь | Prob = 1.00

--- AFTER HYPOTHESES + REWARDS ---
Success rate: 100.0
Average time (successful only): -5.3
Avg path length: 4.0

--- Strategy ---
Готов к переводу → Оформлен | Prob = 1.00
Новый → Рассмотрение рекрутером | Prob = 1.00
Рассмотрение рекрутером → Готов к переводу | Prob = 1.00


## Genetic

In [56]:
# --- Genetic Algorithm ---
def run_genetic_algorithm(transitions_df, terminal_states, transition_times, fitness_fn, population_size=100, generations=50):
    transition_dict = transitions_df.groupby("from")["to"].apply(list).to_dict()

    def generate_path():
        path, current = ["Новый"], "Новый"
        for _ in range(10):
            if current not in transition_dict:
                break
            next_state = random.choice(transition_dict[current])
            path.append(next_state)
            if next_state in terminal_states:
                break
            current = next_state
        return path

    population = [generate_path() for _ in range(population_size)]
    for _ in range(generations):
        scored = [(p, fitness_fn(p, transition_times)) for p in population]
        top = sorted(scored, key=lambda x: x[1], reverse=True)[:population_size // 2]
        offspring = []
        while len(offspring) < population_size // 2:
            p1, p2 = random.sample(top, 2)
            split = random.randint(1, min(len(p1[0]), len(p2[0])) - 1)
            child = p1[0][:split] + [s for s in p2[0][split:] if s not in p1[0][:split]]
            if child[-1] not in terminal_states:
                child = generate_path()
            offspring.append(child)
        population = [x[0] for x in top] + offspring
    return [(p, fitness_fn(p, transition_times)) for p in population]

def fitness_basic(path, transition_times):
    score = 0
    for i in range(len(path) - 1):
        time = transition_times.get((path[i], path[i + 1]), None)
        if time is None:
            return -1000
        score -= time
    return score

def fitness_with_bonus(path, transition_times):
    score = fitness_basic(path, transition_times)
    if path[-1] == "Оформлен":
        score += 500
    elif path[-1] in ["Самоотказ", "Не выходит на связь"]:
        score -= 200
    elif path[-1] == "Отклонен":
        score -= 300
    return score

def extract_ga_policy(paths):
    counter = defaultdict(lambda: defaultdict(int))
    for path, _ in paths:
        for i in range(len(path) - 1):
            counter[path[i]][path[i + 1]] += 1
    print("\n--- Strategy ---")
    for s in counter:
        if counter[s]:
            best = max(counter[s], key=counter[s].get)
            print(f"{s} → {best} | Count = {counter[s][best]}")

In [30]:
transition_times = df.groupby(["from", "to"])["time"].mean().to_dict()

# До гипотез
ga_results = run_genetic_algorithm(df, terminal_states, transition_times, fitness_basic)

print("\n--- BEFORE HYPOTHESES ---")
print("Success rate:", calculate_success_rate(ga_results))
print("Average time (successful only):", calculate_avg_time_successful(ga_results, transition_times))
print("Avg path length:", calculate_avg_path_length(ga_results))
extract_ga_policy(ga_results)

# После гипотез
mod_df = df.copy()
mod_df = mod_df[~((mod_df["from"] == "Готов к переводу") & (mod_df["to"] == "Оформлен"))]
mod_transition_times = mod_df.groupby(["from", "to"])["time"].mean().to_dict()

ga_results_mod = run_genetic_algorithm(mod_df, terminal_states, mod_transition_times, fitness_with_bonus)

print("\n--- AFTER HYPOTHESES ---")
print("Success rate:", calculate_success_rate(ga_results_mod))
print("Average time (successful only):", calculate_avg_time_successful(ga_results_mod, mod_transition_times))
print("Avg path length:", calculate_avg_path_length(ga_results_mod))
extract_ga_policy(ga_results_mod)


--- BEFORE HYPOTHESES ---
Success rate: 0.0
Average time (successful only): None
Avg path length: 3.0

--- Strategy ---
Новый → Рассмотрение рекрутером | Count = 100
Рассмотрение рекрутером → Не выходит на связь | Count = 100

--- AFTER HYPOTHESES ---
Success rate: 100.0
Average time (successful only): 1270.47
Avg path length: 10.89

--- Strategy ---
Новый → Рассмотрение рекрутером | Count = 100
Рассмотрение рекрутером → Интервью | Count = 100
Интервью → Заполнение анкеты | Count = 98
Заполнение анкеты → Проверка | Count = 98
Проверка → Решение о найме | Count = 100
Решение о найме → Согласование офера в компании | Count = 100
Согласование офера в компании → Готов к оформлению | Count = 100
Готов к оформлению → Оформление | Count = 99
Оформление → Оформлен | Count = 99


## Исходное состояние процесса:

На старте анализа процесс подбора показывал высокую склонность к раннему отсеву.

Все три алгоритма (Q-learning, Cross-Entropy, Genetic) в изначальной конфигурации предпочитали короткие траектории, ведущие к “Отклонен” или “Не выходит на связь”.

Это подтверждалось нулевым уровнем успешности (0% “Оформлен”) во всех моделях.

Анализируя граф переходов, я построила идеальный путь, к которому процесс должен стремиться:

Новый → Рекрутер → Заказчик → Интервью → Анкета → Проверка → Решение о найме → Готов к оформлению → Оформление → Оформлен

Однако в логах таких траекторий практически не было.

Поэтому я ввела следующие гипотезы:

	1. Запретить переход “Рекрутер → Отклонен”
    
	2. Удалить зацикливание “Интервью → Интервью”
    
	3. Убрать “Оценка кандидата” — как лишний шаг
    
	4. Запретить переход "Готов к переводу" - "Оформлен"
    
И добавила награды/штрафы в модели:
	+500 за “Оформлен”, +100 за “Решение”, –300 за “Отклонен”, –200 за “Самоотказ” и т.д.

## После гипотез и добавления наград:
Все алгоритмы начали доходить до “Оформлен”

GA оказался наиболее реалистичным: он восстанавливает длинную и логичную последовательность шагов

Q-learning и CEM адаптировались быстрее, но выбирали минимально возможный путь к успеху

Выводы по неэффективностям:

	1.	Слишком агрессивный отсев на этапе “Рекрутер”
    Почти 100% отклонений приходилось на этот шаг
    После запрета — резко выросло количество переходов к интервью
    
	2.	Зацикливание на “Интервью”
    Реальные данные показали много повторов, что снижало эффективность
    После удаления — улучшилась стратегия Qlearning
    
	3.	Недостаточное использование “Решения о найме”
    Этот шаг игнорировался, пока я не добавила награду
    
	4.	Переход “Отклонен” использовался как “по умолчанию”
	Алгоритмы автоматически его выбирали без должной оценки потерь

# RL методы (PPO, TD3)

### Определение среды

In [57]:
pip install gymnasium 

Note: you may need to restart the kernel to use updated packages.


In [58]:
# Кастомная Gym-среда
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class MaskedProcessEnv(gym.Env):
    def __init__(self, transitions_df, reward_map=None, max_steps=50):
        super(MaskedProcessEnv, self).__init__()

        self.transitions_df = transitions_df
        self.reward_map = reward_map or {}
        self.max_steps = max_steps
        self.current_step = 0

        self.states = sorted(set(transitions_df['from']).union(set(transitions_df['to'])))
        self.state_to_idx = {s: i for i, s in enumerate(self.states)}
        self.idx_to_state = {i: s for s, i in self.state_to_idx.items()}

        self.observation_space = spaces.Dict({
            "state": spaces.Discrete(len(self.states)),
            "action_mask": spaces.MultiBinary(len(self.states))
        })
        self.action_space = spaces.Discrete(len(self.states))

        self.transition_dict = transitions_df.groupby("from")["to"].apply(list).to_dict()
        self.transition_times = transitions_df.groupby(["from", "to"])["time"].mean().to_dict()
        self.terminal_states = ["Оформлен", "Отклонен", "Самоотказ", "Не выходит на связь"]

        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_state = "Новый"
        self.current_step = 0
        return self._get_obs(), {}

    def _get_obs(self):
        mask = np.zeros(len(self.states), dtype=np.int8)
        valid_actions = self.transition_dict.get(self.current_state, [])
        for to_state in valid_actions:
            mask[self.state_to_idx[to_state]] = 1
        return {"state": self.state_to_idx[self.current_state], "action_mask": mask}

    def step(self, action_idx):
        self.current_step += 1
        next_state = self.idx_to_state[action_idx]

        valid = next_state in self.transition_dict.get(self.current_state, [])

        if not valid:
            reward = -1000
            terminated = True
            truncated = False
            return self._get_obs(), reward, terminated, truncated, {}

        base_reward = self.reward_map.get(next_state, 0)
        terminated = next_state in self.terminal_states
        truncated = self.current_step >= self.max_steps

        self.current_state = next_state
        return self._get_obs(), base_reward, terminated, truncated, {}

    def render(self):
        print(f"Current state: {self.current_state}")

## Костомная реализация PPO

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import trange

reward_map = {
    "Оформлен": 500,
    "Решение о найме": 400,
    "Интервью": 100,
    "Готов к оформлению": 100,
    "Резерв": -100,
    "Не выходит на связь": -300,
    "Отклонен": -500,
    "Самоотказ": -400
}

class MaskedPPONetwork(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh()
        )
        self.actor = nn.Linear(64, act_dim)
        self.critic = nn.Linear(64, 1)

    def forward(self, obs_tensor, mask_tensor):
        x = self.shared(obs_tensor)
        logits = self.actor(x)
        masked_logits = logits + (1 - mask_tensor) * -1e9
        action_probs = F.softmax(masked_logits, dim=-1)
        dist = torch.distributions.Categorical(action_probs)
        value = self.critic(x).squeeze(-1)
        return dist, value

class RolloutBuffer:
    def __init__(self):
        self.observations = []
        self.action_masks = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        self.values = []
        self.dones = []

    def store(self, obs, mask, action, reward, log_prob, value, done):
        self.observations.append(obs)
        self.action_masks.append(mask)
        self.actions.append(action)
        self.rewards.append(reward)
        self.log_probs.append(log_prob)
        self.values.append(value)
        self.dones.append(done)

    def clear(self):
        self.__init__()

    def get_tensors(self):
        return (
            torch.tensor(self.observations, dtype=torch.float32),
            torch.tensor(self.action_masks, dtype=torch.float32),
            torch.tensor(self.actions, dtype=torch.long),
            torch.tensor(self.rewards, dtype=torch.float32),
            torch.tensor(self.log_probs, dtype=torch.float32),
            torch.tensor(self.values, dtype=torch.float32),
            torch.tensor(self.dones, dtype=torch.float32)
        )

def compute_returns_and_advantages(rewards, values, dones, gamma=0.99, lam=0.95):
    returns = []
    advs = []
    gae = 0
    next_value = 0
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * next_value * (1 - dones[step]) - values[step]
        gae = delta + gamma * lam * (1 - dones[step]) * gae
        adv = gae
        returns.insert(0, adv + values[step])
        advs.insert(0, adv)
        next_value = values[step]
    return torch.tensor(returns), torch.tensor(advs)

def ppo_update(model, optimizer, buffer, clip_eps=0.2, epochs=5, batch_size=64):
    obs, masks, actions, rewards, old_log_probs, values, dones = buffer.get_tensors()
    returns, advantages = compute_returns_and_advantages(rewards, values, dones)
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    for _ in range(epochs):
        for i in range(0, len(obs), batch_size):
            idx = slice(i, i + batch_size)
            dist, value = model(obs[idx], masks[idx])
            entropy = dist.entropy().mean()
            new_log_prob = dist.log_prob(actions[idx])
            ratio = (new_log_prob - old_log_probs[idx]).exp()
            surr1 = ratio * advantages[idx]
            surr2 = torch.clamp(ratio, 1 - clip_eps, 1 + clip_eps) * advantages[idx]
            policy_loss = -torch.min(surr1, surr2).mean()
            value_loss = F.mse_loss(value, returns[idx])
            loss = policy_loss + 0.5 * value_loss - 0.01 * entropy
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def train_custom_ppo(env, total_timesteps=10000, rollout_len=2048):
    obs_dim = 1 + len(env.state_to_idx)
    act_dim = len(env.state_to_idx)
    model = MaskedPPONetwork(obs_dim, act_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    buffer = RolloutBuffer()

    obs, _ = env.reset()

    for _ in trange(total_timesteps // rollout_len):
        buffer.clear()
        for _ in range(rollout_len):
            obs_vec = np.concatenate([[obs['state']], obs['action_mask']])
            obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
            mask_tensor = torch.tensor([obs['action_mask']], dtype=torch.float32)
            dist, value = model(obs_tensor, mask_tensor)
            action = dist.sample()
            log_prob = dist.log_prob(action)

            action = int(action.item())
            next_obs, _, terminated, truncated, _ = env.step(action)
            next_state_name = env.idx_to_state[next_obs['state']]

            reward = reward_map.get(next_state_name, 0)
            buffer.store(obs_vec, obs['action_mask'], action, reward, log_prob.item(), value.item(), float(terminated or truncated))

            obs = next_obs
            if terminated or truncated:
                obs, _ = env.reset()

        ppo_update(model, optimizer, buffer)

    return model

def simulate_masked_ppo(model, env, n_episodes=5):
    idx_to_state = env.idx_to_state
    results = []
    for _ in range(n_episodes):
        obs, _ = env.reset()
        terminated, truncated = False, False
        path = [idx_to_state[obs["state"]]]
        total_reward = 0
        for _ in range(50):
            obs_vec = np.concatenate([[obs["state"]], obs["action_mask"]])
            obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
            mask_tensor = torch.tensor([obs["action_mask"]], dtype=torch.float32)
            dist, _ = model(obs_tensor.unsqueeze(0), mask_tensor.unsqueeze(0))
            action = int(dist.sample().item())
            obs, reward, terminated, truncated, _ = env.step(action)
            path.append(idx_to_state[obs["state"]])
            total_reward += reward
            if terminated or truncated:
                break
        results.append((path, round(total_reward, 2)))
    return results

def evaluate_success_rate(model, env, n_episodes=100):
    success_count = 0
    for _ in range(n_episodes):
        obs, _ = env.reset()
        terminated, truncated = False, False
        for _ in range(50):
            obs_vec = np.concatenate([[obs["state"]], obs["action_mask"]])
            obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
            mask_tensor = torch.tensor([obs["action_mask"]], dtype=torch.float32)
            dist, _ = model(obs_tensor.unsqueeze(0), mask_tensor.unsqueeze(0))
            action = int(dist.sample().item())
            obs, _, terminated, truncated, _ = env.step(action)
            if terminated or truncated:
                break
        if env.idx_to_state[obs['state']] == 'Оформлен':
            success_count += 1
    return round(success_count / n_episodes * 100, 2)

def get_best_successful_path_info(results, transition_times):
    successful = [(path, reward) for path, reward in results if path[-1] == "Оформлен"]
    if not successful:
        return None, None

    best_path, _ = max(successful, key=lambda x: x[1])
    total_time = sum(transition_times.get((best_path[i], best_path[i+1]), 0) for i in range(len(best_path) - 1))

    return best_path, round(total_time, 2)

In [60]:
env = MaskedProcessEnv(df, reward_map)
model = train_custom_ppo(env, total_timesteps=10000, rollout_len=2048)
results = simulate_masked_ppo(model, env, n_episodes=5)

transition_times = df.groupby(["from", "to"])["time"].mean().to_dict()
best_path, best_time = get_best_successful_path_info(results, transition_times)
success_rate = evaluate_success_rate(model, env, n_episodes=100)

/var/folders/gs/z8qql2l148g9gsv1fxp6tgwc0000gn/T/ipykernel_5698/862419918.py:120: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.41it/s]


In [61]:
if best_path:
    print("Лучшая стратегия:", " → ".join(best_path))
    print("Средняя длительность:", best_time)
else:
    print("Успешных стратегий не найдено.")

print(f"Success Rate: {success_rate}%")

Лучшая стратегия: Новый → Рассмотрение заказчиком → Планирование интервью → Интервью → Интервью → Заполнение анкеты → Заполнение анкеты → Проверка → Решение о найме → Согласование офера в компании → Готов к оформлению → Оформление → Оформлен
Средняя длительность: 2188.31
Success Rate: 82.0%


Модель достигла высокой успешности, однако наблюдается зацикливание на этапе "Интервью" — до 6 повторов в одном процессе, что увеличивает длительность и делает стратегию нереалистичной.

In [62]:
class InterviewLimitedEnv(MaskedProcessEnv):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.interview_count = 0
        self.interview_limit = 3

    def reset(self, *args, **kwargs):
        self.interview_count = 0
        return super().reset(*args, **kwargs)

    def step(self, action_idx):
        next_state_name = self.idx_to_state[action_idx]
        if next_state_name == "Интервью":
            self.interview_count += 1
            if self.interview_count > self.interview_limit:
                reward = -500
                terminated, truncated = True, False
                self.current_step += 1
                return self._get_obs(), reward, terminated, truncated, {}
        return super().step(action_idx)

In [63]:
env_limited = InterviewLimitedEnv(df, reward_map)
model_limited = train_custom_ppo(env_limited)
results_limited = simulate_masked_ppo(model_limited, env_limited)
path, duration = get_best_successful_path_info(results_limited, transition_times)
success_rate = evaluate_success_rate(model_limited, env_limited)

print("Гипотеза: Интервью ≤ 3")
print("Лучшая стратегия:", " → ".join(path))
print("Длительность:", duration)
print("Success Rate:", success_rate)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.46it/s]

Гипотеза: Интервью ≤ 3
Лучшая стратегия: Новый → Рассмотрение заказчиком → Интервью → Заполнение анкеты → Проверка → Решение о найме → Согласование офера в компании → Готов к оформлению → Оформление → Оформлен
Длительность: 1653.05
Success Rate: 85.0


Ограничение было реализовано в виде новой среды InterviewLimitedEnv, в которой количество повторений "Интервью" фиксировалось и после превышения порога — процесс завершался со штрафом.

После введения ограничения стратегия стала более короткой, реалистичной и эффективной по времени. Уменьшение Success Rate на ~5% компенсируется улучшением качества пути.

### Вывод по неэффективностям и улучшениям:

	1.	Ключевая неэффективность выявлена на этапе "Интервью" — склонность агента к зацикливанию, особенно в условиях отсутствия ограничений.
    
	2.	Введение гипотезы по ограничению количества повторов улучшило стратегическую логичность и снизило длительность.
    
	3.	PPO в кастомной реализации показал наилучшую адаптацию к процессу, по сравнению с ранее протестированными методами (Q-learning, CEM, Genetic).

## Кастомная реализация TD3

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import trange

reward_map = {
    "Оформлен": 500,
    "Решение о найме": 400,
    "Интервью": 200,
    "Готов к оформлению": 100,
    "Резерв": -200,
    "Не выходит на связь": -300,
    "Отклонен": -500,
    "Самоотказ": -400
}

class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, act_dim), nn.Softmax(dim=-1)
        )

    def forward(self, obs):
        return self.net(obs)

class Critic(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.q = nn.Sequential(
            nn.Linear(obs_dim + act_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, obs, act):
        x = torch.cat([obs, act], dim=-1)
        return self.q(x)

class ReplayBuffer:
    def __init__(self, size=100000):
        self.buffer = []
        self.max_size = size

    def add(self, transition):
        self.buffer.append(transition)
        if len(self.buffer) > self.max_size:
            self.buffer.pop(0)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size)
        batch = [self.buffer[i] for i in indices]
        return map(np.array, zip(*batch))

def train_td3(env, transition_times, reward_map, episodes=500, batch_size=64, gamma=0.99):
    obs_dim = 1 + len(env.state_to_idx)
    act_dim = len(env.state_to_idx)

    actor = Actor(obs_dim, act_dim)
    critic1 = Critic(obs_dim, act_dim)
    critic2 = Critic(obs_dim, act_dim)
    target_actor = Actor(obs_dim, act_dim)
    target_critic1 = Critic(obs_dim, act_dim)
    target_critic2 = Critic(obs_dim, act_dim)

    target_actor.load_state_dict(actor.state_dict())
    target_critic1.load_state_dict(critic1.state_dict())
    target_critic2.load_state_dict(critic2.state_dict())

    actor_opt = torch.optim.Adam(actor.parameters(), lr=1e-3)
    critic1_opt = torch.optim.Adam(critic1.parameters(), lr=1e-3)
    critic2_opt = torch.optim.Adam(critic2.parameters(), lr=1e-3)
    buffer = ReplayBuffer()

    for episode in trange(episodes):
        obs, _ = env.reset()
        obs_vec = np.concatenate([[obs['state']], obs['action_mask']])
        total_reward = 0
        for _ in range(50):
            obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
            with torch.no_grad():
                probs = actor(obs_tensor).numpy()[0] * obs['action_mask']
                probs /= probs.sum() if probs.sum() > 0 else 1
                action = np.random.choice(len(probs), p=probs)

            next_obs, _, terminated, truncated, _ = env.step(action)
            next_vec = np.concatenate([[next_obs['state']], next_obs['action_mask']])
            next_state_name = env.idx_to_state[next_obs['state']]
            reward = reward_map.get(next_state_name, 0)

            buffer.add((obs_vec, action, reward, next_vec, float(terminated or truncated)))
            obs_vec = next_vec
            obs = next_obs
            total_reward += reward

            if terminated or truncated:
                break

            if len(buffer.buffer) >= batch_size:
                o, a, r, o2, d = buffer.sample(batch_size)
                o = torch.tensor(o, dtype=torch.float32)
                a = torch.tensor(a, dtype=torch.long).unsqueeze(1)
                r = torch.tensor(r, dtype=torch.float32).unsqueeze(1)
                o2 = torch.tensor(o2, dtype=torch.float32)
                d = torch.tensor(d, dtype=torch.float32).unsqueeze(1)

                with torch.no_grad():
                    a2 = target_actor(o2)
                    a2 = F.one_hot(a2.argmax(dim=-1), num_classes=act_dim).float()
                    q1_target = target_critic1(o2, a2)
                    q2_target = target_critic2(o2, a2)
                    q_target = r + gamma * (1 - d) * torch.min(q1_target, q2_target)

                a_onehot = F.one_hot(a.squeeze(1), num_classes=act_dim).float()
                q1 = critic1(o, a_onehot)
                q2 = critic2(o, a_onehot)
                loss1 = F.mse_loss(q1, q_target)
                loss2 = F.mse_loss(q2, q_target)

                critic1_opt.zero_grad(); loss1.backward(); critic1_opt.step()
                critic2_opt.zero_grad(); loss2.backward(); critic2_opt.step()

                pred_act = actor(o)
                pred_onehot = F.one_hot(pred_act.argmax(dim=-1), num_classes=act_dim).float()
                loss_actor = -critic1(o, pred_onehot).mean()

                actor_opt.zero_grad(); loss_actor.backward(); actor_opt.step()

                for t, s in zip(target_actor.parameters(), actor.parameters()):
                    t.data.copy_(0.995 * t.data + 0.005 * s.data)
                for t, s in zip(target_critic1.parameters(), critic1.parameters()):
                    t.data.copy_(0.995 * t.data + 0.005 * s.data)
                for t, s in zip(target_critic2.parameters(), critic2.parameters()):
                    t.data.copy_(0.995 * t.data + 0.005 * s.data)

    return actor

def simulate_td3(actor, env, transition_times, n_episodes=5):
    results = []
    for _ in range(n_episodes):
        obs, _ = env.reset()
        obs_vec = np.concatenate([[obs['state']], obs['action_mask']])
        path = [env.idx_to_state[obs['state']]]
        total_reward = 0
        for _ in range(50):
            obs_tensor = torch.tensor([obs_vec], dtype=torch.float32)
            with torch.no_grad():
                probs = actor(obs_tensor).numpy()[0] * obs['action_mask']
                probs /= probs.sum() if probs.sum() > 0 else 1
                action = np.random.choice(len(probs), p=probs)
            obs, reward, terminated, truncated, _ = env.step(action)
            path.append(env.idx_to_state[obs['state']])
            obs_vec = np.concatenate([[obs['state']], obs['action_mask']])
            total_reward += reward
            if terminated or truncated:
                break
        results.append((path, round(total_reward, 2)))
    return results

def analyze_results(results, transition_times):
    successful = [(p, r) for p, r in results if p[-1] == 'Оформлен']
    if not successful:
        return None, None, 0.0
    best_path, _ = max(successful, key=lambda x: x[1])
    duration = sum(transition_times.get((best_path[i], best_path[i+1]), 0) for i in range(len(best_path)-1))
    success_rate = round(len(successful) / len(results) * 100, 2)
    return best_path, round(duration, 2), success_rate


In [66]:
actor = train_td3(env, transition_times, reward_map, episodes=500)
results = simulate_td3(actor, env, transition_times, n_episodes=10)
best_path, duration, success_rate = analyze_results(results, transition_times)

print("Лучшая стратегия:", " → ".join(best_path))
print("Длительность:", duration)
print("Success Rate:", success_rate, "%")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 57.73it/s]

Лучшая стратегия: Новый → Рассмотрение заказчиком → Планирование интервью → Интервью → Интервью → Интервью → Планирование интервью → Интервью → Планирование интервью → Интервью → Заполнение анкеты → Проверка → Решение о найме → Согласование офера в компании → Готов к оформлению → Оформление → Оформлен
Длительность: 2769.47
Success Rate: 10.0 %


Агент демонстрирует знание процесса, но склонен к глубокому зацикливанию, особенно на этапах "Интервью" и "Заполнение анкеты". 

In [71]:
# Ограничения на 'Интервью', 'Анкета', и исключение 'Оценка кандидата'
class CombinedLimitedEnv(MaskedProcessEnv):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.interview_count = 0
        self.anketa_count = 0
        self.interview_limit = 3
        self.anketa_limit = 2

    def reset(self, *args, **kwargs):
        self.interview_count = 0
        self.anketa_count = 0
        return super().reset(*args, **kwargs)

    def step(self, action_idx):
        next_state_name = self.idx_to_state[action_idx]

        if next_state_name == "Интервью":
            self.interview_count += 1
            if self.interview_count > self.interview_limit:
                reward = -500
                terminated, truncated = True, False
                self.current_step += 1
                return self._get_obs(), reward, terminated, truncated, {}

        if next_state_name == "Заполнение анкеты":
            self.anketa_count += 1
            if self.anketa_count > self.anketa_limit:
                reward = -500
                terminated, truncated = True, False
                self.current_step += 1
                return self._get_obs(), reward, terminated, truncated, {}

        if next_state_name == "Оценка кандидата":
            reward = -500
            terminated, truncated = True, False
            self.current_step += 1
            return self._get_obs(), reward, terminated, truncated, {}

        return super().step(action_idx)


In [ ]:
env_limited = CombinedLimitedEnv(df, reward_map)
actor = train_td3(env_limited, transition_times, reward_map)
results = simulate_td3(actor, env_limited, transition_times)
path, duration, success_rate = analyze_results(results, transition_times)

print("Гипотеза: Интервью ≤ 3 и Анкета ≤ 2")
if path is not None:
    print("Лучшая стратегия:", " → ".join(path))
    print("Длительность:", duration)
    print("Success Rate:", success_rate)
else:
    print("Успешных стратегий не найдено.")
    print("Success Rate:", success_rate)

 10%|█████████████▏                                                                                                                 | 52/500 [00:00<00:05, 77.09it/s]

Ограничения были реализованы через CombinedLimitedEnv:

	1. "Интервью" не более 3 раз
	2. "Заполнение анкеты" не более 2 раз
	3. "Оценка кандидата" полностью исключается как неэффективный путь

После применения гипотез поведение агента стало более логичным и кратким, но Success Rate снизился. Это говорит о том, что TD3 требует больше данных или дообучения, чтобы адаптироваться к новым ограничениям.

### Вывод по TD3:

	1.	TD3 способен находить глубокие и длинные стратегии, но при этом склонен к зацикливанию.
	2.	Без ограничений агент часто переиспользует одни и те же шаги, особенно "Интервью" и "Анкета".
	3.	Введение гипотез по ограничениям дало улучшение длительности, но Success Rate стал ещё более чувствительным к выбору пути.
	4.	TD3 требует большего количества эпизодов или гибридного обучения (например, с imitation learning от PPO), чтобы стабилизировать выбор стратегии.